<a href="https://colab.research.google.com/github/iwa103/MintPy/blob/main/2_%E3%82%B0%E3%83%A9%E3%83%95%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Geotiffから抽出データのCSVファイルからのグラフ作成（グラフ点数が多い場合に有効）

## 基準日：20200303　毎回確認必要！！

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
from google.colab import files
import io

# ファイルをアップロードし、結果を別の変数名に格納
uploaded_files = files.upload()

# CSVファイルを読み込む
main_file_name = 'geoTIFF_values.csv'
columns_file_name = 'columns.csv'

main_data = pd.read_csv(io.BytesIO(uploaded_files[main_file_name]))
columns_data = pd.read_csv(io.BytesIO(uploaded_files[columns_file_name]))

# 日付列の処理
main_data['Date'] = pd.to_datetime(main_data['Date'].str.split('_').str[1], format='%Y%m%d')
base_date = pd.to_datetime('2020-03-03')  # 基準日を20200303に設定

# グラフを保存するディレクトリを作成
os.makedirs('plots', exist_ok=True)

# 指定された各列に対してグラフを描画し、保存
for column_number in columns_data['Column']:
    column_name = f'{column_number}_value'
    plt.figure(figsize=(10, 6))
    plt.scatter(main_data['Date'], main_data[column_name], alpha=0.5)

    # 基準日に青い垂直線を描画
    plt.axvline(x=base_date, color='blue', linestyle='--')

    # 近似線の計算
    x = (main_data['Date'] - base_date).dt.days  # 基準日からの経過日数を使用
    y = main_data[column_name]

    # 傾きを計算（切片はゼロとする）
    slope = np.polyfit(x, y, 1)[0]

    # 傾きのみを使用して近似線を描画
    plt.plot(main_data['Date'], slope * x, "r--")

    plt.title(f'Scatter Plot of {column_name} Over Time')
    plt.xlabel('Date')
    plt.ylabel(column_name)
    plt.grid(True)
    plt.gcf().autofmt_xdate()
    plt.savefig(f'plots/{column_name}.png')
    plt.close()

# すべてのPNGファイルをZIP形式で圧縮
with zipfile.ZipFile('plots.zip', 'w') as zipf:
    for root, dirs, files in os.walk('plots'):
        for file in files:
            zipf.write(os.path.join(root, file))

# ZIPファイルをダウンロード
#files.download('plots.zip')
